In [4]:
from ibl_pipeline import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior
from ibl_pipeline import subject, acquisition, action
from ibl_pipeline.utils import psychofit as psy
import numpy as np
import datetime
import datajoint as dj
import json
import pandas as pd
from uuid import UUID

In [2]:
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [5]:
subj = subject.Subject & {'subject_uuid': UUID('8f6fc7ce-4de0-4b8b-973b-72fd70145b9c')}
# get water and date
water_info_query = (action.WaterAdministration & subj).proj(
    'water_administered', 'watertype_name', water_date='DATE(administration_time)')
water_info = water_info_query.fetch(as_dict=True)
water_info = pd.DataFrame(water_info)

In [6]:
water_types = water_info.watertype_name.unique()

In [7]:
water_types

array(['Water', 'Water 10% Sucrose', 'Hydrogel', 'Water 2% Citric Acid'],
      dtype=object)

In [8]:
water_info.pop('administration_time')
water_info.pop('subject_uuid');

In [43]:
water_info_type = water_info.pivot_table(
    index='water_date', columns='watertype_name', values='water_administered', aggfunc='sum')

In [44]:
water_info_type = water_info_type.where((pd.notnull(water_info_type)), None)

In [31]:
x

[nan,
 None,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.14,
 nan,
 nan,
 nan,
 nan,
 0.7,
 0.8,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.43,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [45]:
weight_info_query = (action.Weighing & subj).proj(
    'weight', weighing_date='DATE(weighing_time)')

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:151: UserWarning:

Mysql server has gone away. Reconnecting to the server.



In [46]:
weight_info = weight_info_query.fetch(as_dict=True)
weight_info = pd.DataFrame(weight_info)

In [47]:
weight_info.pop('subject_uuid');

In [48]:
weight_info.pop('weighing_time');

In [49]:
weight_info = weight_info.where((pd.notnull(weight_info)), None)

In [53]:
data = [
    go.Bar(
        x = [t.strftime('%Y-%m-%d') for t in water_info_type.index.tolist()],
        y = water_info_type[water_type].tolist(),
        name = water_type,
        yaxis = 'y1')
    for water_type in water_types
]

data.append(
    go.Scatter(
        x = [t.strftime('%Y-%m-%d') for t in weight_info['weighing_date'].tolist()],
        y = weight_info['weight'].tolist(),
        mode = 'lines+markers',
        name = 'Weight',
        marker = dict(
            size = 6,
            color = 'black'),
        yaxis = 'y2'
    ))

layout = go.Layout(
    yaxis=dict(
        title='Water intake (mL)',
    ),
    yaxis2=dict(
        title='Weight (g)',
        overlaying = 'y',
        side='right'
    ),
    width=600,
    height=400,
    title='Water intake and weight', 
    xaxis={'title':'Date'}, 
    legend=dict(
        x=0, 
        y=1.2,
        orientation='h'),
    barmode='stack'
)

In [51]:
fig = go.Figure(data=data, layout=layout)

In [52]:
plotly.offline.iplot(fig)

In [39]:
f = open("water_weight.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()